<a href="https://colab.research.google.com/github/Harshit28012002/MRM-Harshit-JIo-insititute/blob/main/CLV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime

# Load the dataset
file_path = '/content/Online_Sales.csv'  # Update with your file path
df = pd.read_csv(file_path)

# Convert Transaction_Date to datetime
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], format='%d-%m-%Y')

# Aggregate data to calculate RFM metrics
# Define the current date as a reference for recency calculation
current_date = df['Transaction_Date'].max()

# Group data by CustomerID
rfm = df.groupby('CustomerID').agg({
    'Transaction_Date': lambda x: (current_date - x.max()).days,  # Recency
    'Transaction_ID': 'nunique',  # Frequency
    'Avg_Price': lambda x: (x * df.loc[x.index, 'Quantity']).sum()  # Monetary
}).reset_index()

# Rename columns
rfm.rename(columns={
    'Transaction_Date': 'Recency',
    'Transaction_ID': 'Frequency',
    'Avg_Price': 'Monetary'
}, inplace=True)

# Add RFM scores
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])  # 4: Most recent, 1: Least recent
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])  # 4: Most frequent
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])  # 4: Highest monetary value

# Combine RFM scores into a single RFM_Segment
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

# Label customer segments based on RFM score
def rfm_segment(df):
    if df['RFM_Score'] >= 9:
        return 'Champions'
    elif df['RFM_Score'] >= 7:
        return 'Loyal Customers'
    elif df['RFM_Score'] >= 5:
        return 'Potential Loyalists'
    else:
        return 'Needs Attention'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

# Display the RFM dataframe
import ace_tools as tools; tools.display_dataframe_to_user(name="RFM Analysis with Segments", dataframe=rfm)


ValueError: time data "1/1/2019" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/Online_Sales.csv'
df = pd.read_csv(file_path)

# Convert Transaction_Date to datetime
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], format='%d-%m-%Y')

# Calculate the total price for each transaction
df['Total_Price'] = (df['Quantity'] * df['Avg_Price']) + df['Delivery_Charges']

# Calculate RFM metrics
# Recency: Days since the last purchase
recency_df = df.groupby('CustomerID').Transaction_Date.max().reset_index()
recency_df.columns = ['CustomerID', 'LastPurchaseDate']
recency_df['Recency'] = (pd.to_datetime('today') - recency_df['LastPurchaseDate']).dt.days

# Frequency: Total number of transactions
frequency_df = df.groupby('CustomerID').Transaction_ID.nunique().reset_index()
frequency_df.columns = ['CustomerID', 'Frequency']

# Monetary: Total spending
monetary_df = df.groupby('CustomerID').Total_Price.sum().reset_index()
monetary_df.columns = ['CustomerID', 'Monetary']

# Merge RFM metrics into a single dataframe
rfm_df = recency_df.merge(frequency_df, on='CustomerID').merge(monetary_df, on='CustomerID')

# Rank customers on Recency, Frequency, and Monetary values
rfm_df['R_Score'] = pd.qcut(rfm_df['Recency'], 4, ['1', '2', '3', '4'])
rfm_df['F_Score'] = pd.qcut(rfm_df['Frequency'], 4, ['4', '3', '2', '1'])
rfm_df['M_Score'] = pd.qcut(rfm_df['Monetary'], 4, ['4', '3', '2', '1'])

# Combine R, F, M scores to get RFM score
rfm_df['RFM_Score'] = rfm_df[['R_Score', 'F_Score', 'M_Score']].agg(''.join, axis=1)

# Interpret RFM Score
rfm_df['Customer_Segment'] = rfm_df['RFM_Score'].replace({
    '111': 'Best Customers',
    '112': 'Loyal Customers',
    '113': 'Potential Loyalists',
    '123': 'Recent Customers',
    '124': 'Promising',
    '134': 'Need Attention',
    '144': 'About to Sleep',
    '222': 'At Risk',
    '233': 'Hibernating',
    '244': 'Lost'
})

# Display the RFM dataframe
print(rfm_df.head())


ValueError: time data "1/1/2019" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# Convert Transaction_Date to datetime with format inference
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], dayfirst=True)


ValueError: time data "1/13/2019" doesn't match format "%d/%m/%Y", at position 12. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# Attempt to convert with dayfirst=True and coerce errors
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], format='%d/%m/%Y', errors='coerce')

# Check for rows where conversion failed
invalid_dates = df[df['Transaction_Date'].isna()]

# If invalid dates exist, try converting using another format
if not invalid_dates.empty:
    df.loc[df['Transaction_Date'].isna(), 'Transaction_Date'] = pd.to_datetime(
        df.loc[df['Transaction_Date'].isna(), 'Transaction_Date'],
        format='%m/%d/%Y',
        errors='coerce'
    )

# Check again for any remaining invalid dates
remaining_invalid_dates = df[df['Transaction_Date'].isna()]
if not remaining_invalid_dates.empty:
    print("There are still some invalid dates. Here are the rows:")
    print(remaining_invalid_dates)


There are still some invalid dates. Here are the rows:
       CustomerID  Transaction_ID Transaction_Date     Product_SKU  \
1607        13468           17754              NaT  GGOEAAQB036114   
1608        13468           17754              NaT  GGOEAAQB036115   
1609        13468           17754              NaT  GGOEGAFB035817   
1610        13468           17755              NaT  GGOEGBFC018799   
1611        13468           17755              NaT  GGOEGDHB072199   
...           ...             ...              ...             ...   
52919       14410           48493              NaT  GGOENEBB078899   
52920       14410           48494              NaT  GGOEGAEB091117   
52921       14410           48495              NaT  GGOENEBQ084699   
52922       14600           48496              NaT  GGOENEBQ079199   
52923       14600           48497              NaT  GGOENEBQ079099   

                                   Product_Description Product_Category  \
1607                     Andr

In [ ]:
# Drop rows with invalid Transaction_Date
df_cleaned = df.dropna(subset=['Transaction_Date'])

# Continue with the RFM analysis using df_cleaned


In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv('/content/Online_Sales.csv')

# Convert Transaction_Date to datetime
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce')

# Drop rows with invalid Transaction_Date
df = df.dropna(subset=['Transaction_Date'])

# Calculate the total price for each transaction
df['Total_Price'] = df['Quantity'] * df['Avg_Price']

# Calculate RFM values
# Reference date for recency calculation (e.g., the latest transaction date in the dataset)
reference_date = df['Transaction_Date'].max() + pd.Timedelta(days=1)

# Group by CustomerID and calculate RFM metrics
rfm = df.groupby('CustomerID').agg({
    'Transaction_Date': lambda x: (reference_date - x.max()).days,  # Recency
    'Transaction_ID': 'nunique',                                   # Frequency
    'Total_Price': 'sum'                                            # Monetary
})

# Rename columns to RFM metrics
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Display the RFM table
print(rfm.head())

# Optional: Create RFM segments based on quantiles
rfm['R'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

# Combine RFM segments into a single score
rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)

# Display the RFM segments
print(rfm.head())

# Now, you can interpret the RFM segments for CLV analysis


            Recency  Frequency  Monetary
CustomerID                              
12346           108          1     30.99
12347            60         31  13834.90
12348            74          8   1442.12
12350            18         11   1360.07
12356           108         13   1442.47
            Recency  Frequency  Monetary  R  F  M RFM_Segment  RFM_Score
CustomerID                                                              
12346           108          1     30.99  3  1  1         311          5
12347            60         31  13834.90  3  4  4         344         11
12348            74          8   1442.12  3  2  2         322          7
12350            18         11   1360.07  4  2  2         422          8
12356           108         13   1442.47  3  3  2         332          8


In [ ]:
import numpy as np

# Define discount rate and time horizon
r = 0.10  # Discount rate (e.g., 10%)
N = 12    # Time horizon in months

# Function to calculate CLV
def calculate_clv(frequency, monetary, discount_rate, periods):
    avg_revenue_per_period = monetary / frequency
    discounted_revenue = [
        avg_revenue_per_period / ((1 + discount_rate) ** t)
        for t in range(1, periods + 1)
    ]
    return np.sum(discounted_revenue)

# Add CLV calculation to the RFM DataFrame
rfm['CLV'] = rfm.apply(
    lambda row: calculate_clv(row['Frequency'], row['Monetary'], r, N), axis=1
)

# Display the RFM table with CLV
print(rfm.head())


            Recency  Frequency  Monetary  R  F  M RFM_Segment  RFM_Score  \
CustomerID                                                                 
12346           108          1     30.99  3  1  1         311          5   
12347            60         31  13834.90  3  4  4         344         11   
12348            74          8   1442.12  3  2  2         322          7   
12350            18         11   1360.07  4  2  2         422          8   
12356           108         13   1442.47  3  3  2         332          8   

                    CLV  
CustomerID               
12346        211.156310  
12347       3040.862742  
12348       1228.270156  
12350        842.463440  
12356        756.042003  


In [ ]:
import pandas as pd
import numpy as np

# Load your CSV file
df = pd.read_csv('/content/Online_Sales.csv')

# Convert Transaction_Date to datetime
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'], errors='coerce')

# Drop rows with invalid Transaction_Date
df = df.dropna(subset=['Transaction_Date'])

# Calculate the total price for each transaction
df['Total_Price'] = df['Quantity'] * df['Avg_Price']

# Calculate RFM values
reference_date = df['Transaction_Date'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'Transaction_Date': lambda x: (reference_date - x.max()).days,  # Recency
    'Transaction_ID': 'nunique',                                   # Frequency
    'Total_Price': 'sum'                                            # Monetary
})
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Calculate CLV using the formula provided
r = 0.10  # Discount rate (e.g., 10%)
N = 12    # Time horizon in months

def calculate_clv(frequency, monetary, discount_rate, periods):
    avg_revenue_per_period = monetary / frequency
    discounted_revenue = [
        avg_revenue_per_period / ((1 + discount_rate) ** t)
        for t in range(1, periods + 1)
    ]
    return np.sum(discounted_revenue)

rfm['CLV'] = rfm.apply(
    lambda row: calculate_clv(row['Frequency'], row['Monetary'], r, N), axis=1
)

# Optional: Create RFM segments based on quantiles
rfm['R'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1])
rfm['F'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=[1, 2, 3, 4])
rfm['M'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4])

rfm['RFM_Segment'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis=1)

# Display the RFM table with CLV
print(rfm.head())


            Recency  Frequency  Monetary          CLV  R  F  M RFM_Segment  \
CustomerID                                                                   
12346           108          1     30.99   211.156310  3  1  1         311   
12347            60         31  13834.90  3040.862742  3  4  4         344   
12348            74          8   1442.12  1228.270156  3  2  2         322   
12350            18         11   1360.07   842.463440  4  2  2         422   
12356           108         13   1442.47   756.042003  3  3  2         332   

            RFM_Score  
CustomerID             
12346               5  
12347              11  
12348               7  
12350               8  
12356               8  


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
file_path = '/content/bank marketing - logistic (1) (1).xlsx'
df = pd.read_excel(file_path)

# Identify categorical variables
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Convert categorical variables to dummy variables
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Separate the independent variables (X) and the dependent variable (y)
X = df.drop(columns=['y'])
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0)  # Convert 'yes'/'no' to 1/0

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)


Accuracy: 0.901657458563536
Confusion Matrix:
[[789  18]
 [ 71  27]]
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       807
           1       0.60      0.28      0.38        98

    accuracy                           0.90       905
   macro avg       0.76      0.63      0.66       905
weighted avg       0.88      0.90      0.88       905

